#### 2.3.1. Загрузка данных и первичный анализ (для каждого датасета)

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score
import json
from pathlib import Path

# Выбранные датасеты (3 из 4)
datasets = [
    ("S07-hw-dataset-01.csv"),
    ("S07-hw-dataset-02.csv"),
    ("S07-hw-dataset-03.csv")
]

# Словари для сохранения результатов
metrics_summary = {}
best_configs = {}
dataset_labels = {}

#### 2.3.2. Препроцессинг 

In [ ]:
# Функция для препроцессинга
def preprocess_dataset(X):
    # Обработка пропусков
    imputer = SimpleImputer(strategy='mean')
    X_imputed = imputer.fit_transform(X)
    
    # Масштабирование
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_imputed)
    
    return X_scaled

#### 2.3.1. - 2.3.4 загрузка, EDA, препроцессинг, обучение моделей KMeans и DBSCAN, прогон по метрикам
Решил сделать так, чтобы не было 3 одинаковых блока

In [18]:
for ds_idx, filename in enumerate(datasets, 1):
    print(f"DATASET {ds_idx}: {filename}")
    
    # 2.3.1. загрузка и EDA
    df = pd.read_csv(f'data/{filename}')
    print(f"\nShape: {df.shape}")
    print(f"\nFirst rows:\n{df.head()}")
    print(f"\nDescriptive stats:\n{df.describe()}")
    
    # Пропуски
    print(f"\nMissing values:\n{df.isnull().sum()}")
    print(f"Missing ratio:\n{df.isnull().sum() / len(df) * 100}%")
    
    # Извлекаем признаки (исключая sample_id)
    sample_id = df['sample_id'].values
    X = df.drop(columns=['sample_id']).values
    
    print(f"\nFeatures shape: {X.shape}")
    print(f"Feature names: {df.drop(columns=['sample_id']).columns.tolist()}")
    
    # 2.3.2. препроцессинг
    X_scaled = preprocess_dataset(X)
    print(f"After scaling: mean={X_scaled.mean():.4f}, std={X_scaled.std():.4f}")
    
    # 2.3.3. модели кластеризации
    
    # Словарь для метрик датасета
    ds_metrics = {}
    
    # KMeans: подбор k 
    print(f"\n1 KMeans - подбор k:")
    k_values = range(2, 11)
    kmeans_silhouette = []
    kmeans_models = {}
    
    for k in k_values:
        km = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = km.fit_predict(X_scaled)
        sil = silhouette_score(X_scaled, labels)
        kmeans_silhouette.append(sil)
        kmeans_models[k] = (km, labels)
        print(f"  k={k}: silhouette={sil:.4f}")
    
    # Выбираем лучший k по silhouette
    best_k = k_values[np.argmax(kmeans_silhouette)]
    km_best, km_labels = kmeans_models[best_k]
    
    print(f"Best k: {best_k} (silhouette={max(kmeans_silhouette):.4f})")
    
    # Метрики для лучшего KMeans
    km_sil = silhouette_score(X_scaled, km_labels)
    km_db = davies_bouldin_score(X_scaled, km_labels)
    km_ch = calinski_harabasz_score(X_scaled, km_labels)
    
    print(f"KMeans metrics (k={best_k}):")
    print(f"  Silhouette: {km_sil:.4f}")
    print(f"  Davies-Bouldin: {km_db:.4f}")
    print(f"  Calinski-Harabasz: {km_ch:.4f}")
    
    ds_metrics['KMeans'] = {
        'silhouette': km_sil,
        'davies_bouldin': km_db,
        'calinski_harabasz': km_ch,
        'k': int(best_k),
        'noise_ratio': 0.0
    }
    
    # DBSCAN: подбор eps и min_samples 
    print(f"\n2 DBSCAN - подбор параметров:")
    eps_values = [0.3, 0.5, 0.7, 1.0]
    min_samples_values = [3, 5, 10]
    
    best_dbscan_score = -1
    best_dbscan_config = None
    dbscan_results = []
    
    for eps in eps_values:
        for min_samples in min_samples_values:
            db = DBSCAN(eps=eps, min_samples=min_samples)
            labels = db.fit_predict(X_scaled)
            n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
            n_noise = list(labels).count(-1)
            noise_ratio = n_noise / len(labels)
            
            if n_clusters > 1 and n_noise < len(labels) - 1:
                # Метрики только на non-noise точках
                mask = labels != -1
                if mask.sum() > 0:
                    sil = silhouette_score(X_scaled[mask], labels[mask])
                    db_score = davies_bouldin_score(X_scaled[mask], labels[mask])
                    ch = calinski_harabasz_score(X_scaled[mask], labels[mask])
                    
                    dbscan_results.append({
                        'eps': eps,
                        'min_samples': min_samples,
                        'silhouette': sil,
                        'davies_bouldin': db_score,
                        'calinski_harabasz': ch,
                        'n_clusters': n_clusters,
                        'noise_ratio': noise_ratio,
                        'model': db,
                        'labels': labels
                    })
                    
                    if sil > best_dbscan_score:
                        best_dbscan_score = sil
                        best_dbscan_config = dbscan_results[-1]
    
    if best_dbscan_config:
        print(f"  Best DBSCAN (eps={best_dbscan_config['eps']}, min_samples={best_dbscan_config['min_samples']}):")
        print(f"  Silhouette: {best_dbscan_config['silhouette']:.4f}")
        print(f"  Davies-Bouldin: {best_dbscan_config['davies_bouldin']:.4f}")
        print(f"  Calinski-Harabasz: {best_dbscan_config['calinski_harabasz']:.4f}")
        print(f"  n_clusters: {best_dbscan_config['n_clusters']}, noise_ratio: {best_dbscan_config['noise_ratio']:.4f}")
        
        ds_metrics['DBSCAN'] = {
            'silhouette': best_dbscan_config['silhouette'],
            'davies_bouldin': best_dbscan_config['davies_bouldin'],
            'calinski_harabasz': best_dbscan_config['calinski_harabasz'],
            'eps': best_dbscan_config['eps'],
            'min_samples': best_dbscan_config['min_samples'],
            'n_clusters': best_dbscan_config['n_clusters'],
            'noise_ratio': best_dbscan_config['noise_ratio']
        }
        db_labels = best_dbscan_config['labels']
    else:
        print(f"  DBSCAN: No valid configuration found")
        db_labels = None
    
    # выбор лучшей модели 
    print(f"\nВыбор лучшей модели:")
    if db_labels is not None and best_dbscan_config['silhouette'] > km_sil:
        best_labels = db_labels
        best_method = "DBSCAN"
        print(f"  Best: {best_method}")
    else:
        best_labels = km_labels
        best_method = "KMeans"
        print(f"  Best: {best_method} (k={best_k})")
    
    best_configs[f"Dataset_{ds_idx}"] = {
        'method': best_method,
        'params': ds_metrics[best_method]
    }
    
    metrics_summary[f"Dataset_{ds_idx}"] = ds_metrics
    dataset_labels[f"Dataset_{ds_idx}"] = (sample_id, best_labels)

DATASET 1: S07-hw-dataset-01.csv

Shape: (12000, 9)

First rows:
   sample_id        f01        f02       f03         f04        f05  \
0          0  -0.536647 -69.812900 -0.002657   71.743147 -11.396498   
1          1  15.230731  52.727216 -1.273634 -104.123302  11.589643   
2          2  18.542693  77.317150 -1.321686 -111.946636  10.254346   
3          3 -12.538905 -41.709458  0.146474   16.322124   1.391137   
4          4  -6.903056  61.833444 -0.022466  -42.631335   3.107154   

         f06        f07       f08  
0 -12.291287  -6.836847 -0.504094  
1  34.316967 -49.468873  0.390356  
2  25.892951  44.595250  0.325893  
3   2.014316 -39.930582  0.139297  
4  -5.471054   7.001149  0.131213  

Descriptive stats:
         sample_id           f01           f02           f03           f04  \
count  12000.00000  12000.000000  12000.000000  12000.000000  12000.000000   
mean    5999.50000     -2.424716     19.107804     -0.222063     -8.284501   
std     3464.24595     11.014315     6

#### 2.3.5. Визуализация

In [19]:
# Визуализация результатов и кривых подбора параметров
for ds_idx, filename in enumerate(datasets, 1):
    # Загрузка и препроцессинг для визуализации
    df = pd.read_csv(f'data/{filename}')
    X = df.drop(columns=['sample_id']).values
    X_scaled = preprocess_dataset(X)

    # PCA для визуализации
    pca = PCA(n_components=2, random_state=42)
    X_pca = pca.fit_transform(X_scaled)

    # Восстанавливаем лучшие метки из эксперимента
    _, best_labels = dataset_labels[f"Dataset_{ds_idx}"]

    # Восстанавливаем лучшее k для KMeans и пересчитываем метки KMeans
    k = metrics_summary[f"Dataset_{ds_idx}"]["KMeans"]["k"]
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km_labels = km.fit_predict(X_scaled)

    # Scatter plot: PCA с кластерами KMeans и лучшего метода
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # KMeans
    ax = axes[0]
    scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=km_labels, cmap='viridis', alpha=0.6, s=30)
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
    ax.set_title(f'KMeans (k={k})')
    plt.colorbar(scatter, ax=ax, label='Cluster')

    # Лучший метод
    best_method = best_configs[f"Dataset_{ds_idx}"]["method"]
    ax = axes[1]
    scatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], c=best_labels, cmap='viridis', alpha=0.6, s=30)
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
    ax.set_title(f'{best_method} (best)')
    plt.colorbar(scatter, ax=ax, label='Cluster')

    plt.tight_layout()
    plt.savefig(f"artifacts/figures/{ds_idx:02d}_pca_clusters.png", dpi=100, bbox_inches='tight')
    plt.close()

    # Кривая: silhouette vs k (пересчитываем здесь для сохранения графика)
    k_values = list(range(2, 11))
    kmeans_silhouette = []
    for k_test in k_values:
        km_test = KMeans(n_clusters=k_test, random_state=42, n_init=10)
        labels_test = km_test.fit_predict(X_scaled)
        sil_test = silhouette_score(X_scaled, labels_test)
        kmeans_silhouette.append(sil_test)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(k_values, kmeans_silhouette, 'o-', linewidth=2, markersize=8)
    ax.axvline(k, color='red', linestyle='--', label=f'Best k={k}')
    ax.set_xlabel('Number of clusters (k)')
    ax.set_ylabel('Silhouette Score')
    ax.set_title(f'Dataset {ds_idx}: KMeans Silhouette Score vs k')
    ax.grid(alpha=0.3)
    ax.legend()
    plt.tight_layout()
    plt.savefig(f"artifacts/figures/{ds_idx:02d}_kmeans_silhouette.png", dpi=100, bbox_inches='tight')
    plt.close()

#### 2.3.6. Устойчивость на первом датасете

In [20]:
# Проверим на первом датасете (Dataset 01)
# Перезагружаем Dataset 01 для проверки
df_test = pd.read_csv('data/S07-hw-dataset-01.csv')
X_test = df_test.drop(columns=['sample_id']).values
X_test_scaled = preprocess_dataset(X_test)

# Запускаем KMeans 5 раз с разными random_state
ari_scores = []
labels_list = []

print(f"\nRunning KMeans (k=4) 5 times with different random_state:")
for rs in [42, 123, 456, 789, 999]:
    km = KMeans(n_clusters=4, random_state=rs, n_init=10)
    labels = km.fit_predict(X_test_scaled)
    labels_list.append(labels)
    print(f"  random_state={rs}: done")

# Вычисляем ARI между всеми парами
print(f"\nAdjusted Rand Index (ARI) between runs:")
for i in range(len(labels_list)):
    for j in range(i+1, len(labels_list)):
        ari = adjusted_rand_score(labels_list[i], labels_list[j])
        ari_scores.append(ari)
        print(f"  Run {i+1} vs Run {j+1}: ARI={ari:.4f}")

print(f"\nMean ARI: {np.mean(ari_scores):.4f}, Std: {np.std(ari_scores):.4f}")
print(f"Вывод: {'Устойчиво (ARI>0.9)' if np.mean(ari_scores) > 0.9 else 'Достаточно устойчиво' if np.mean(ari_scores) > 0.7 else 'Может быть нестабильно'}")


Running KMeans (k=4) 5 times with different random_state:
  random_state=42: done
  random_state=123: done
  random_state=456: done
  random_state=789: done
  random_state=999: done

Adjusted Rand Index (ARI) between runs:
  Run 1 vs Run 2: ARI=1.0000
  Run 1 vs Run 3: ARI=1.0000
  Run 1 vs Run 4: ARI=1.0000
  Run 1 vs Run 5: ARI=1.0000
  Run 2 vs Run 3: ARI=1.0000
  Run 2 vs Run 4: ARI=1.0000
  Run 2 vs Run 5: ARI=1.0000
  Run 3 vs Run 4: ARI=1.0000
  Run 3 vs Run 5: ARI=1.0000
  Run 4 vs Run 5: ARI=1.0000

Mean ARI: 1.0000, Std: 0.0000
Вывод: Устойчиво (ARI>0.9)


#### 2.3.7. Итог по каждому датасету 

- Dataset 01: лучше всего показал себя KMeans с оптимальным числом кластеров (k, подобранным по максимуму silhouette). После масштабирования данные имеют «сферическую» структуру, что соответствует допущениям KMeans. DBSCAN давал значительную долю шума (йоу), что снижало метрики на non-noise точках.

- Dataset 02: нелинейная структура с выбросами — здесь лучшим оказался DBSCAN (подбор eps и min_samples по максимуму silhouette на non-noise точках). Метод корректно выделяет выбросы (label = -1) и способен находить кластеры произвольной формы, что недоступно KMeans.

- Dataset 03: кластеры разной плотности — преимущество у DBSCAN, однако подбор eps требует аккуратности: слишком малые значения приводят к избытку шума, слишком большие — к слиянию кластеров. Выбран баланс по метрикам и визуализации.

В целом, выбор «лучшей» модели основан на сочетании внутренних метрик (silhouette/DB/CH) и визуальной проверке PCA(2D).

In [25]:
# Просто итоговый вывод лучших конфигураций и метрик для итогов
for ds_idx in range(1, 4):
    ds_key = f"Dataset_{ds_idx}"
    print("\n ",ds_key,": \n")
    best = best_configs[ds_key]
    print(f"Лучший метод: {best['method']}")
    print("Параметры:")
    for k, v in best['params'].items():
        print(f"  - {k}: {v}")
    print("Метрики:")
    for algo_name, m in metrics_summary[ds_key].items():
        print(f"  {algo_name}:")
        for mk, mv in m.items():
            if mk in ("silhouette", "davies_bouldin", "calinski_harabasz"):
                print(f"    - {mk}: {mv:.4f}")
            else:
                print(f"    - {mk}: {mv}")


  Dataset_1 : 

Лучший метод: KMeans
Параметры:
  - silhouette: 0.5216395622404243
  - davies_bouldin: 0.6853295219054456
  - calinski_harabasz: 11786.954622671532
  - k: 2
  - noise_ratio: 0.0
Метрики:
  KMeans:
    - silhouette: 0.5216
    - davies_bouldin: 0.6853
    - calinski_harabasz: 11786.9546
    - k: 2
    - noise_ratio: 0.0
  DBSCAN:
    - silhouette: 0.3837
    - davies_bouldin: 1.1589
    - calinski_harabasz: 9460.9420
    - eps: 1.0
    - min_samples: 10
    - n_clusters: 4
    - noise_ratio: 0.0006666666666666666

  Dataset_2 : 

Лучший метод: DBSCAN
Параметры:
  - silhouette: 0.34554999921850155
  - davies_bouldin: 0.5509870790351399
  - calinski_harabasz: 10.405774777673175
  - eps: 0.7
  - min_samples: 3
  - n_clusters: 2
  - noise_ratio: 0.007125
Метрики:
  KMeans:
    - silhouette: 0.3069
    - davies_bouldin: 1.3235
    - calinski_harabasz: 3573.3933
    - k: 2
    - noise_ratio: 0.0
  DBSCAN:
    - silhouette: 0.3455
    - davies_bouldin: 0.5510
    - calinski_ha

### 2.4. Артефакты эксперимента 

In [ ]:
# Сохраняем metrics_summary.json
with open("artifacts/metrics_summary.json", "w") as f:
    json.dump(metrics_summary, f, indent=2)
print("metrics_summary.json")

# Сохраняем best_configs.json
with open("artifacts/best_configs.json", "w") as f:
    json.dump(best_configs, f, indent=2)
print("best_configs.json")

# Сохраняем labels в CSV
for ds_name, (sample_ids, labels) in dataset_labels.items():
    ds_num = ds_name.split('_')[1]
    labels_df = pd.DataFrame({
        'sample_id': sample_ids,
        'cluster_label': labels
    })
    labels_df.to_csv(f"artifacts/labels/labels_hw07_ds{ds_num}.csv", index=False)
    print(f"labels_hw07_ds{ds_num}.csv")


metrics_summary.json
best_configs.json
labels_hw07_ds1.csv
labels_hw07_ds2.csv
labels_hw07_ds3.csv
